# Cartesian Bining

Note that this is specifically for the FLUKA USRBIN energy deposition with XYZ binning and a single detector per output unit. 

Before running, make sure to change the file type from .bnn.lis to .txt and update all the UPDATE sections.

In [ ]:
# Set the file path and the new file path as needed

### UPDATE ###
file_path = "hfatigue_Tungsten.txt"
new_file_path = "hfatigue_Tungsten_formated.csv"

output=[]
with open(file_path,"r") as file:
  for line in file:
    output.append(line.rstrip('\n'))
print(output[:4])

In [ ]:
# get binning parameters from the file

x_min=float(output[2].split()[3])
x_max=float(output[2].split()[5])
x_bin=int(output[2].split()[7])
x_del=float(output[2].split()[10])

y_min=float(output[3].split()[3])
y_max=float(output[3].split()[5])
y_bin=int(output[3].split()[7])
y_del=float(output[3].split()[10])

z_min=float(output[4].split()[3])
z_max=float(output[4].split()[5])
z_bin=int(output[4].split()[7])
z_del=float(output[4].split()[10])

In [ ]:
# split the output into data and error lists

cut=output.index("      Percentage errors follow in a matrix A(ix,iy,iz), format (1(5x,1p,10(1x,e11.4)))")

data_raw=output[8:(cut-1)]
error_raw=output[(cut+2):]

data_aver=[]
data_max = []
error=[]

### UPDATE ###
tot_protons = 1.25e12 # the number of total protons/cycle is specific to the experiment and should be adjusted accordingly
cycle_time = 0.0481 # the cycle length in seconds
act_time = 0.0431 # we apply all energy during a limited period of time, which is the active time in seconds

for line in data_raw:
  for datpoint in line.split():
    data_aver.append(float(datpoint)*1.6e-10*tot_protons/cycle_time)
    data_max.append(float(datpoint)*1.6e-10*tot_protons/act_time)

for line in error_raw:
  for datpoint in line.split():
    error.append(float(datpoint))

In [ ]:
# create a single csv file in the following format:
# | Min x for a bin | Min y for a bin | Min z for a bin | Average Power Deposited | Max Power Deposited | Error % |

def sci(number):
  return str("{:e}".format(number))

import csv

with open(new_file_path, 'w', newline='') as file:
    output_f = csv.writer(file)
    captions = ["X [cm]", "Y [cm]", "Z [cm]", "Average Power [W/cm^3]", "Max Power [W/cm^3]", "Error [%]"]
    output_f.writerow(captions)

    n = 0
    for i_z in range(0, z_bin):
      for i_y in range(0, y_bin):
        for i_x in range(0, x_bin):
          row = [sci(x_min+i_x*x_del), sci(y_min+i_y*y_del), sci(z_min+i_z*z_del), sci(data_aver[n]), sci(data_max[n]), sci(error[n])]
          output_f.writerow(row)
          n += 1

# Axisymmetric Bining

This section is specifically for the FLUKA USRBIN energy deposition with R-Z-$\Phi$ binning, 1 detector per output unit, and only 1 bin per $\Phi$. 

Before running, make sure to change the file type from .bnn.lis to .txt and update all the UPDATE sections.

In [ ]:
# Set the file path and the new file path as needed

### UPDATE ###
file_path = "hfatigue_Tungsten.txt"
new_file_path = "hfatigue_Tungsten_formated.csv"

output=[]
with open(file_path,"r") as file:
  for line in file:
    output.append(line.rstrip('\n'))
print(output[:4])

In [ ]:
# get binning parameters from the file

r_min=float(output[2].split()[3])
r_max=float(output[2].split()[5])
r_bin=int(output[2].split()[7])
r_del=float(output[2].split()[10])

z_min=float(output[3].split()[3])
z_max=float(output[3].split()[5])
z_bin=int(output[3].split()[7])
z_del=float(output[3].split()[10])

x_cent=float(output[4].split()[4].rstrip(','))
y_cent=float(output[4].split()[7])

In [ ]:
# split the output into data and error lists

cut=output.index("      Percentage errors follow in a matrix A(ir,iz), format (1(5x,1p,10(1x,e11.4)))")

data_raw=output[7:(cut-1)] #CHECK number 7 here must be the first data raw
error_raw=output[(cut+2):]

data_aver=[]
data_max = []
error=[]

### UPDATE ###
tot_protons = 1.25e12 # the number of total protons/cycle is specific to the experiment and should be adjusted accordingly
cycle_time = 0.0481 # the cycle length in seconds
act_time = 0.0431 # we apply all energy during a limited period of time, which is the active time in seconds

for line in data_raw:
  for datpoint in line.split():
    data_aver.append(float(datpoint)*1.6e-10*tot_protons/cycle_time) # note the conversion from GeV/cm^3/primary into J/cm^3
    data_max.append(float(datpoint)*1.6e-10*tot_protons/act_time) # note the conversion from GeV/cm^3/primary into J/cm^3

for line in error_raw:
  for datpoint in line.split():
    error.append(float(datpoint))

In [ ]:
# create a single csv file in the following format:
# | Min x for a bin | Min y for a bin | Min z for a bin | Average Power Deposited | Max Power Deposited | Error % |

def sci(number):
  return str("{:e}".format(number))

import csv

with open(new_file_path, 'w', newline='') as file:
    output_f = csv.writer(file)
    captions = ["R [cm]", "Z [cm]", "Average Power [W/cm^3]", "Max Power [W/cm^3]", "Error [%]"]
    output_f.writerow(captions)

    n = 0
    for i_z in range(0, z_bin):
      for i_r in range(0, r_bin):
        row = [sci(r_min+i_r*r_del), sci(z_min+i_z*z_del), sci(data_aver[n]), sci(data_max[n]), sci(error[n])]
        output_f.writerow(row)
        n += 1